# **Part 1**

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize # the minimize method to find the optimum weights for the portfolio
import yfinance as yf

## Fetch Crypto Data

In [ ]:
# Step 1: Get Historical Data for Cryptocurrencies
# Choose four cryptocurrencies of your choice
tickers = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD']

In [ ]:
# Define the time frame
start_date = '2022-11-01'
end_date = '2023-11-01'

In [ ]:
# Download historical data
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
data, type(data)

[*********************100%%**********************]  4 of 4 completed


(Ticker           BTC-USD      ETH-USD    LTC-USD   XRP-USD
 Date                                                      
 2022-11-01  20485.273438  1579.704590  55.127190  0.464577
 2022-11-02  20159.503906  1519.711792  60.558468  0.450874
 2022-11-03  20209.988281  1531.541748  61.921314  0.455135
 2022-11-04  21147.230469  1645.093384  67.596596  0.504404
 2022-11-05  21282.691406  1627.968018  69.721199  0.493330
 ...                  ...          ...        ...       ...
 2023-10-27  33909.800781  1780.045288  67.055794  0.546107
 2023-10-28  34089.574219  1776.618164  67.782776  0.545391
 2023-10-29  34538.480469  1795.546021  68.800842  0.556394
 2023-10-30  34502.363281  1810.088623  69.250839  0.578944
 2023-10-31  34667.781250  1816.458984  68.949562  0.600282
 
 [365 rows x 4 columns],
 pandas.core.frame.DataFrame)

## Define Initial Cap and Random Weights

In [ ]:
# Step 2: Define Initial Capital and Coefficients
initial_capital = 1000
coefficients = np.array([0.4, 0.18, 0.27, 0.15])  # Sum of coefficients must be equal to 1

In [ ]:
assert round(sum(coefficients)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in coefficients), "All values are greater than 0."

## The Buy and Hold Strategy Implementation

In [ ]:
def buy_and_hold_strategy_backtest(asset_returns, initial_cap=1000):
  account_balance = initial_cap
  cumulative_return = (1 + asset_returns).cumprod()
  account_balance *= cumulative_return

  return account_balance

In [ ]:
# Step 2: Calculate Daily Returns
close_data = data.copy()
data = data.pct_change().dropna()
data

Ticker,BTC-USD,ETH-USD,LTC-USD,XRP-USD
Date,,,,
2022-11-02,-0.015903,-0.037977,0.098523,-0.029496
2022-11-03,0.002504,0.007784,0.022505,0.009451
2022-11-04,0.046375,0.074142,0.091653,0.108251
2022-11-05,0.006406,-0.010410,0.031431,-0.021955
2022-11-06,-0.016737,-0.034235,-0.023453,-0.045933
...,...,...,...,...
2023-10-27,-0.007227,-0.013300,-0.026122,-0.013683
2023-10-28,0.005302,-0.001925,0.010841,-0.001311
2023-10-29,0.013168,0.010654,0.015020,0.020174


In [ ]:
close_data

Ticker,BTC-USD,ETH-USD,LTC-USD,XRP-USD
Date,,,,
2022-11-01,20485.273438,1579.704590,55.127190,0.464577
2022-11-02,20159.503906,1519.711792,60.558468,0.450874
2022-11-03,20209.988281,1531.541748,61.921314,0.455135
2022-11-04,21147.230469,1645.093384,67.596596,0.504404
2022-11-05,21282.691406,1627.968018,69.721199,0.493330
...,...,...,...,...
2023-10-27,33909.800781,1780.045288,67.055794,0.546107
2023-10-28,34089.574219,1776.618164,67.782776,0.545391
2023-10-29,34538.480469,1795.546021,68.800842,0.556394


## Sharpe Ratio Impelementation

In [ ]:
def Sharpe_ratio(backtest_results):
  temp_df = pd.DataFrame(index=backtest_results.index)
  temp_df['Balances'] = backtest_results['Balances']
  returns = temp_df['Balances'].pct_change().dropna()
  risk_free_rate = (1.02 ** (1 / 360)) - 1
  return (returns.mean() - risk_free_rate) / returns.std()

## Sharpe Ratio Objective Function

In [ ]:
# Step 3: Define Objective Function
def Sharpe_ratio_objective(weights, cryptos):
    crypto_capitals = [weight * initial_capital for weight in weights]

    crypto1_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[0]], crypto_capitals[0])
    crypto2_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[1]], crypto_capitals[1])
    crypto3_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[2]], crypto_capitals[2])
    crypto4_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[3]], crypto_capitals[3])

    backtest_balances = crypto1_bh + crypto2_bh + crypto3_bh + crypto4_bh
    backtest_balances = backtest_balances.to_frame()
    backtest_balances.columns = ['Balances']
    return -Sharpe_ratio(backtest_balances)  # We minimize the negative Sharpe ratio to maximize the Sharpe ratio


Sharpe Ratio for Buy and Hold with Initial Weights

In [ ]:
init_sr = -1 * Sharpe_ratio_objective(coefficients, data)
init_sr

0.04502073952504293

## Find Optimum Weights for Maximizing the Sharpe Ratio

In [ ]:
# Step 5: Define Constraints
n = len(coefficients)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
for i in range(n):
    constraints.append({'type': 'ineq', 'fun': lambda x,i=i: x[i] - 1e-10})


In [ ]:
bounds = [(0, None)] * n

In [ ]:
# Step 6: Optimize Portfolio for Sharpe Ratio
result = minimize(Sharpe_ratio_objective, coefficients, args=(data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_sharpe = result.x

In [ ]:
np.set_printoptions(precision=15)
optimal_weights_sharpe

array([9.999999996999998e-01, 9.999964875259670e-11,
       1.000003370041358e-10, 1.000002580742176e-10])

In [ ]:
assert round(sum(optimal_weights_sharpe)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_sharpe), "All values are greater than 0."

Calculate Sharpe Ratio for Buy and Hold with optimum weights



In [ ]:
optimum_sr = -1 * Sharpe_ratio_objective(optimal_weights_sharpe, data)
optimum_sr

0.07061678668857137

## Sortino Ratio Implementation

In [ ]:
def downside_deviation(backtest_results):
    temp_df = pd.DataFrame(index=backtest_results.index)
    temp_df['Balances'] = backtest_results['Balances']
    returns = temp_df['Balances'].pct_change().dropna()
    downside_returns = returns[returns < 0]
    return downside_returns.std()

def Sortino_ratio(backtest_results):
    temp_df = pd.DataFrame(index=backtest_results.index)
    temp_df['Balances'] = backtest_results['Balances']
    returns = temp_df['Balances'].pct_change().dropna()
    risk_free_rate = (1.02 ** (1 / 360)) - 1
    return (returns.mean() - risk_free_rate) / downside_deviation(backtest_results)


## Sortino Ratio Objective Function

In [ ]:
def Sortino_ratio_objective(weights, cryptos):
    crypto_capitals = [weight * initial_capital for weight in weights]

    crypto1_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[0]], crypto_capitals[0])
    crypto2_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[1]], crypto_capitals[1])
    crypto3_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[2]], crypto_capitals[2])
    crypto4_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[3]], crypto_capitals[3])

    backtest_balances = crypto1_bh + crypto2_bh + crypto3_bh + crypto4_bh
    backtest_balances = backtest_balances.to_frame()
    backtest_balances.columns = ['Balances']
    return -Sortino_ratio(backtest_balances)  # We minimize the negative Sortino ratio to maximize the Sortino ratio


Sortino Ratio for Buy and Hold with initial weights

In [ ]:
init_sortino = -1 * Sortino_ratio_objective(coefficients, data)
init_sortino

0.06155679890556221

## Find Optimum Weights for Maximizing the Sortino Ratio

In [ ]:
# Step 5: Define Constraints
n = len(coefficients)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
for i in range(n):
    constraints.append({'type': 'ineq', 'fun': lambda x,i=i: x[i] - 1e-10})

In [ ]:
result = minimize(Sortino_ratio_objective, coefficients, args=(data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_sortino = result.x
optimal_weights_sortino

array([9.999999996999994e-01, 1.000002199103012e-10,
       1.000007264495562e-10, 9.999971684049314e-11])

In [ ]:
assert round(sum(optimal_weights_sortino)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_sortino), "All values are greater than 0."

Calculate Sortino Ratio for Buy and Hold with optimum weights


In [ ]:
optimum_sortino = -1 * Sortino_ratio_objective(optimal_weights_sortino, data)
optimum_sortino

0.10083459939218704

## Net Profit Implementation

In [ ]:
def compute_net_profit(initial_investment, final_portfolio_value):
    net_profit = final_portfolio_value - initial_investment
    return net_profit


In [ ]:
def net_profit_objective(weights, cryptos):
    crypto_capitals = [weight * initial_capital for weight in weights]

    crypto1_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[0]], crypto_capitals[0])
    crypto2_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[1]], crypto_capitals[1])
    crypto3_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[2]], crypto_capitals[2])
    crypto4_bh = buy_and_hold_strategy_backtest(cryptos[cryptos.columns[3]], crypto_capitals[3])

    backtest_balances = crypto1_bh + crypto2_bh + crypto3_bh + crypto4_bh
    final_portfolio_value = crypto1_bh[-1] + crypto2_bh[-1] + crypto3_bh[-1] + crypto4_bh[-1]

    backtest_balances = backtest_balances.to_frame()
    backtest_balances.columns = ['Balances']
    return -compute_net_profit(initial_capital, final_portfolio_value)

Net Profit for Buy and Hold with initial weights

In [ ]:
init_net = -1 * net_profit_objective(coefficients, data)
init_net

415.422261975481

## Find Optimum Weights for Maximizing the Net Profit

In [ ]:
n = len(coefficients)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
for i in range(n):
    constraints.append({'type': 'ineq', 'fun': lambda x,i=i: x[i] - 1e-10})

In [ ]:
result = minimize(net_profit_objective, coefficients, args=(data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_netProfit = result.x
optimal_weights_netProfit

array([9.999999993446324e-01, 3.172978546572836e-10,
       1.893683543308100e-10, 1.487014122947272e-10])

In [ ]:
assert round(sum(optimal_weights_netProfit)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_netProfit), "All values are greater than 0."

Calculate Net Profit for Buy and Hold with optimum weights


In [ ]:
optimum_net = -1 * net_profit_objective(optimal_weights_netProfit, data)
optimum_net

692.3269952589274

# **Part 2**

In [ ]:
# Define the time frame
start_date = '2023-11-02'
end_date = '2023-12-02'

In [ ]:
# Step1: Download historical data
test_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
test_data, type(test_data)

[*********************100%%**********************]  4 of 4 completed


(Ticker           BTC-USD      ETH-USD    LTC-USD   XRP-USD
 Date                                                      
 2023-11-02  34938.242188  1800.620972  69.394196  0.606439
 2023-11-03  34732.324219  1832.795166  69.491142  0.613002
 2023-11-04  35082.195312  1857.698608  70.472099  0.616184
 2023-11-05  35049.355469  1894.157715  71.662651  0.661147
 2023-11-06  35037.371094  1899.837402  74.482864  0.715517
 2023-11-07  35443.562500  1888.124268  73.564384  0.686402
 2023-11-08  35655.277344  1889.322388  73.179939  0.688076
 2023-11-09  36693.125000  2120.561035  73.752213  0.667475
 2023-11-10  37313.968750  2078.289795  73.305000  0.660575
 2023-11-11  37138.050781  2052.713867  75.174309  0.662735
 2023-11-12  37054.519531  2045.187012  74.891273  0.661536
 2023-11-13  36502.355469  2055.265381  71.491547  0.671524
 2023-11-14  35537.640625  1979.052612  70.635262  0.629673
 2023-11-15  37880.582031  2060.408447  74.080322  0.649648
 2023-11-16  36154.769531  1960.881592  

In [ ]:
# Step 2: Calculate Daily Returns
test_close_data = test_data.copy()
test_data = test_data.pct_change().dropna()
test_data

Ticker,BTC-USD,ETH-USD,LTC-USD,XRP-USD
Date,,,,
2023-11-03,-0.005894,0.017868,0.001397,0.010822
2023-11-04,0.010073,0.013588,0.014116,0.005191
2023-11-05,-0.000936,0.019626,0.016894,0.072970
2023-11-06,-0.000342,0.002999,0.039354,0.082236
2023-11-07,0.011593,-0.006165,-0.012331,-0.040691
2023-11-08,0.005973,0.000635,-0.005226,0.002439
2023-11-09,0.029108,0.122392,0.007820,-0.029940
2023-11-10,0.016920,-0.019934,-0.006064,-0.010337
2023-11-11,-0.004715,-0.012306,0.025500,0.003270


## Sharpe Ratio Calculation on Buy and Hold with data from 2023-11-02 till 2023-12-02

Calculate Sharpe Ratio for Buy and Hold with initial Weights


In [ ]:
init_sr = -1 * Sharpe_ratio_objective(coefficients, test_data)
init_sr

0.1285905855559773

Calculate Sharpe Ratio for Buy and Hold with optimum Weights from 2022-11-01 till 2023-11-01


In [ ]:
optimum_sr = -1 * Sharpe_ratio_objective(optimal_weights_sharpe, test_data)
optimum_sr

0.17728326095656688

## Sortino Ratio Calculation on Buy and Hold with data from 2023-11-02 till 2023-12-02

Calculate Sortino Ratio for Buy and Hold with initial Weights

In [ ]:
init_sortino = -1 * Sortino_ratio_objective(coefficients, test_data)
init_sortino

0.1781280722317655

Calculate Sortino Ratio for Buy and Hold with optimum weights from 2022-11-01 till 2023-11-01

In [ ]:
optimum_sortino = -1 * Sortino_ratio_objective(optimal_weights_sortino, test_data)
optimum_sortino

0.2517768331859311

## Net Profit Calculation on Buy and Hold with data from 2023-11-02 till 2023-12-02




Calculate Net Profit for Buy and Hold with initial Weights

In [ ]:
init_net = -1 * net_profit_objective(coefficients, test_data)
init_net

81.55078648430435

Calculate Net Profit for Buy and Hold with optimum weights from 2022-11-01 till 2023-11-01

In [ ]:
optimum_net = -1 * net_profit_objective(optimal_weights_netProfit, test_data)
optimum_net

107.34678041153438

## Find Optimum Weights for Maximizing the Sharpe Ratio on Test Data

In [ ]:
# Step 6: Optimize Portfolio for Sharpe Ratio
result = minimize(Sharpe_ratio_objective, coefficients, args=(test_data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_sharpe = result.x

In [ ]:
optimal_weights_sharpe

array([8.154934652358800e-01, 1.845065345641201e-01,
       1.000001203805417e-10, 1.000000097919201e-10])

In [ ]:
assert round(sum(optimal_weights_sharpe)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_sharpe), "All values are greater than 0."

Calculate Sharpe Ratio for Buy and Hold with optimum weights from 2023-11-02 till 2023-12-02

In [ ]:
optimum_sr = -1 * Sharpe_ratio_objective(optimal_weights_sharpe, test_data)
optimum_sr

0.17989591604262797

## Find Optimum Weights for Maximizing the Sortino Ratio on Test Data

In [ ]:
result = minimize(Sortino_ratio_objective, coefficients, args=(test_data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_sortino = result.x
optimal_weights_sortino

array([9.999892580658809e-11, 9.999999997000011e-01,
       9.999998745735539e-11, 1.000000724588057e-10])

In [ ]:
assert round(sum(optimal_weights_sortino)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_sortino), "All values are greater than 0."

Calculate Sortino Ratio for Buy and Hold with optimum weights from 2023-11-02 till 2023-12-02

In [ ]:
optimum_sortino = -1 * Sortino_ratio_objective(optimal_weights_sortino, test_data)
optimum_sortino

0.31220518881769305

## Find Optimum Weights for Maximizing the Net Profit on Test Data

In [ ]:
result = minimize(net_profit_objective, coefficients, args=(test_data,), method='SLSQP', constraints=constraints, bounds=bounds)
optimal_weights_netProfit = result.x
optimal_weights_netProfit

array([1.067818611311111e-10, 9.999999997171569e-01,
       9.011741353148750e-11, 8.594394640404346e-11])

In [ ]:
assert round(sum(optimal_weights_netProfit)) == 1, "The sum of coefficients must be equal to one."
assert all(value > 0 for value in optimal_weights_netProfit), "All values are greater than 0."

Calculate Net Profit for Buy and Hold with optimum weights from 2023-11-02 till 2023-12-02

In [ ]:
optimum_net = -1 * net_profit_objective(optimal_weights_netProfit, test_data)
optimum_net

159.12228356278865